# Florida Restaurant Relocation 

### Introduction

A client would like to relocate their gourmet Mexican restaurant to Florida.  They want to live in a beach community on the east coast of Florida but are not interested in south Florida.  The location needs to be a community that can support another gourmet restaurant, and the client wants a community that does not already have too many Mexican restaurants as competition.

### Data

A google search of "florida east coast cities list" led to the website https://sciencetrends.com/map-of-florida-east-coast-beaches-and-cities/ , which contains a table of coastal cities on the east coast of Florida with a region column, that has elements "Northeastern Coast of Florida," "Central East Coast of Florida," and "Southern East Coast of Florida."  The table was scraped eliminating cities in the region "Southern East Coast of Florida."  The remaining cities are added to a list.

In [6]:
CityList

['American Beach',
 'Atlantic Beach',
 'Fernandina Beach',
 'Flagler Beach',
 'Jacksonville Beach',
 'Neptune Beach',
 'Palm Coast',
 'Ponte Vedra',
 'St Augustine',
 'Vilano Beach',
 'Cocoa Beach',
 'Daytona Beach',
 'Indian Harbour',
 'Jensen Beach',
 'Melbourne Beach',
 'New Smyrna Beach',
 'Ormond Beach',
 'Playalinda Beach',
 'Satellite Beach',
 'Stuart',
 'Vero Beach']

Then for each city United States zip code data from the python package “uszipcode” is used to find a list of zip codes and demographic data for each zip code in the city.

In [10]:
res

[SimpleZipcode(zipcode='32233', zipcode_type='Standard', major_city='Atlantic Beach', post_office_city='Atlantic Beach, FL', common_city_list=['Atlantic Beach', 'Atlantic Bch', 'Jacksonville', 'Mayport'], county='Duval County', state='FL', lat=30.36, lng=-81.42, timezone='Eastern', radius_in_miles=3.0, area_code_list=['904'], population=23673, population_density=2323.0, land_area_in_sqmi=10.19, water_area_in_sqmi=3.16, housing_units=10128, occupied_housing_units=8858, median_home_value=211100, median_household_income=50338, bounds_west=-81.459709, bounds_east=-81.390331, bounds_north=30.399843, bounds_south=30.323343)]

Finally, the Foursquare API will be used to find the number of restaurants and the number of Mexican restaurants in each zip code.

In [14]:
results

{'meta': {'code': 200, 'requestId': '5e65a93978a484001b948ad5'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Jacksonville',
  'headerFullLocation': 'Jacksonville',
  'headerLocationGranularity': 'city',
  'query': 'restaurant',
  'totalResults': 78,
  'suggestedBounds': {'ne': {'lat': 30.399843, 'lng': -81.390331},
   'sw': {'lat': 30.323343, 'lng': -81.459709}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '589a87033493554d7a2d026b',
       'name': 'Hawkers Asian Street Fare',
       'location': {'address': '241 Atlantic Blvd',
        'lat': 30.32439,
        'lng': -81.39701,
        'labeledLatLngs': [{'label': 'display',
          

In [2]:
!pip install requests
!pip install beautifulsoup4
!pip install uszipcode

     |████████████████████████████████| 378kB 19.8MB/s eta 0:00:01
     |████████████████████████████████| 204kB 17.8MB/s eta 0:00:01
     |████████████████████████████████| 122kB 29.3MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/2b/dc/d5/e4bc37df332b7a88022bc79d6cdccb79068fdc1620d86afc3a
Successfully built autopep8


In [3]:
import requests
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import json
from pandas.io.json import json_normalize
from uszipcode import SearchEngine, SimpleZipcode, Zipcode

In [4]:
url = "https://sciencetrends.com/map-of-florida-east-coast-beaches-and-cities/"
page = requests.get(url)
print(page.status_code)
soup = BeautifulSoup(page.content, 'html.parser')
#print(soup.prettify())
tb = soup.find('tbody')

200


In [5]:
table_rows = tb.find_all('tr')
#print(table_rows)
CityList=[]
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    if row[0]!='East Coast Florida Beaches' and\
    row[1]!='Southeastern Coast of Florida':
        row[0]=row[0][:-8]
        CityList.append(row[0]) 

In [6]:
CityList

['American Beach',
 'Atlantic Beach',
 'Fernandina Beach',
 'Flagler Beach',
 'Jacksonville Beach',
 'Neptune Beach',
 'Palm Coast',
 'Ponte Vedra',
 'St Augustine',
 'Vilano Beach',
 'Cocoa Beach',
 'Daytona Beach',
 'Indian Harbour',
 'Jensen Beach',
 'Melbourne Beach',
 'New Smyrna Beach',
 'Ormond Beach',
 'Playalinda Beach',
 'Satellite Beach',
 'Stuart',
 'Vero Beach']

In [8]:
search = SearchEngine()

Start downloading data for simple zipcode database, total size 9MB ...
  1 MB finished ...
  2 MB finished ...
  3 MB finished ...
  4 MB finished ...
  5 MB finished ...
  6 MB finished ...
  7 MB finished ...
  8 MB finished ...
  9 MB finished ...
  10 MB finished ...
  Complete!


In [11]:
CLIENT_ID = 'SYX02EPX2C0Y0JRDNE1MN5Q4VPBNM41JTJ4RXJQKOTLWIMAE'
CLIENT_SECRET = 'X5UGUQDAZ35KQZECX3Q1WJFFY2PVLOKLPJTRR1L43JQ2ICUI'
VERSION = '20180605'
LIMIT=200
Food_ID = '4d4b7105d754a06374d81259'
Rests_ID = '4bf58dd8d48988d1c4941735'
Mex_ID = '4bf58dd8d48988d1c1941735'

In [12]:
url = 'https://api.foursquare.com/v2/venues/explore?\
client_id={}&client_secret={}&v={}\
&sw=30.323343,-81.459709&ne=30.399843,-81.390331\
&intent=browse\
&limit={}\
&categoryId={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION,
    LIMIT, Rests_ID)
url

'https://api.foursquare.com/v2/venues/explore?client_id=SYX02EPX2C0Y0JRDNE1MN5Q4VPBNM41JTJ4RXJQKOTLWIMAE&client_secret=X5UGUQDAZ35KQZECX3Q1WJFFY2PVLOKLPJTRR1L43JQ2ICUI&v=20180605&sw=30.323343,-81.459709&ne=30.399843,-81.390331&intent=browse&limit=200&categoryId=4bf58dd8d48988d1c4941735'

In [13]:
results = requests.get(url).json()

In [14]:
results

{'meta': {'code': 200, 'requestId': '5e65a93978a484001b948ad5'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Jacksonville',
  'headerFullLocation': 'Jacksonville',
  'headerLocationGranularity': 'city',
  'query': 'restaurant',
  'totalResults': 78,
  'suggestedBounds': {'ne': {'lat': 30.399843, 'lng': -81.390331},
   'sw': {'lat': 30.323343, 'lng': -81.459709}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '589a87033493554d7a2d026b',
       'name': 'Hawkers Asian Street Fare',
       'location': {'address': '241 Atlantic Blvd',
        'lat': 30.32439,
        'lng': -81.39701,
        'labeledLatLngs': [{'label': 'display',
          

In [9]:
res=search.by_city_and_state("Atlantic Beach", "FL",returns=None)

In [10]:
res

[SimpleZipcode(zipcode='32233', zipcode_type='Standard', major_city='Atlantic Beach', post_office_city='Atlantic Beach, FL', common_city_list=['Atlantic Beach', 'Atlantic Bch', 'Jacksonville', 'Mayport'], county='Duval County', state='FL', lat=30.36, lng=-81.42, timezone='Eastern', radius_in_miles=3.0, area_code_list=['904'], population=23673, population_density=2323.0, land_area_in_sqmi=10.19, water_area_in_sqmi=3.16, housing_units=10128, occupied_housing_units=8858, median_home_value=211100, median_household_income=50338, bounds_west=-81.459709, bounds_east=-81.390331, bounds_north=30.399843, bounds_south=30.323343)]